# k-Nearest-Neighbors: kNN-regressor

## loading data and data exploration

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor

In [2]:
# Load data from scikit_learn
diabetes_dict = load_diabetes()
# Construct DataFrame with features data, Data is already normalized according to documentation
data = pd.DataFrame(diabetes_dict['data'], columns=diabetes_dict['feature_names'])
# Construct DataFrame with target data
data_target = pd.DataFrame(diabetes_dict['target'], columns=['target'])
# Generate train, test split with default 75 % training data, fixed random state
X_train, X_test, y_train, y_test = train_test_split(data, data_target, random_state=1243)